<a href="https://colab.research.google.com/github/fubotz/ICL_2024W/blob/main/FinalProject_Fabian_SCHAMBECK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project: Finetuning a Multilingual Model
Description here...

Model:

Dataset:

In [1]:
!pip install datasets
!pip install bertviz transformers
!pip install evaluate
!pip install scipy
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.2 MB/s eta 0:00:00
   

In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import cosine

# Load the multilingual DistilBERT model and tokenizer
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define example word pairs for cognate detection
word_pairs = [("mother", "mère"),  # English-French cognates
              ("dog", "chien"),   # English-French non-cognates
              ("cat", "chat"),    # English-French cognates
              ("house", "maison") # English-French non-cognates
             ]

# Function to calculate cosine similarity between word embeddings
def calculate_similarity(word1, word2):
    # Tokenize words
    tokens1 = tokenizer(word1, return_tensors="pt")
    tokens2 = tokenizer(word2, return_tensors="pt")

    # Get embeddings
    with torch.no_grad():
        embedding1 = model(**tokens1).last_hidden_state.mean(dim=1)
        embedding2 = model(**tokens2).last_hidden_state.mean(dim=1)

    # Calculate cosine similarity
    similarity = 1 - cosine(embedding1.squeeze(), embedding2.squeeze())
    return similarity

# Compute similarities for each word pair
results = []
for word1, word2 in word_pairs:
    similarity = calculate_similarity(word1, word2)
    results.append((word1, word2, similarity))

results

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

[('mother', 'mère', 0.7876212098923396),
 ('dog', 'chien', 0.7402774221501399),
 ('cat', 'chat', 0.7773961899533575),
 ('house', 'maison', 0.7626993196154054)]

In [ ]:
from datasets import load_dataset

# Load the Europarl dataset for English-French
def load_europarl_data():
    print("Loading Europarl dataset...")
    dataset = load_dataset("Helsinki-NLP/europarl", "en-fr")
    print("Dataset loaded successfully!")

    # Show the first ten entries
    print("\nFirst ten entries from the training set:")
    for i in range(10):
        print(f"Entry {i + 1}:")
        print(f"  English: {dataset['train'][i]['translation']['en']}")
        print(f"  French:  {dataset['train'][i]['translation']['fr']}")

    return dataset

# Call the function to load the dataset
dataset = load_europarl_data()

Loading Europarl dataset...
Dataset loaded successfully!

First ten entries from the training set:
Entry 1:
  English: Resumption of the session
  French:  Reprise de la session
Entry 2:
  English: I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
  French:  Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.
Entry 3:
  English: Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.
  French:  Comme vous avez pu le constater, le grand "bogue de l'an 2000" ne s'est pas produit. En revanche, les citoyens d'un certain nombre de nos pays ont été victimes de 